# Post-traitement des résultats du k-mean

Ce notebook va permettre d'évaluer l'efficacité de plusieurs versions de l'algorithme de post-traitement, en particulier le calcul du mode statistique, pour l'estimation du triplet `(r, g, b)` le plus fréquent dans un voisinage de points.

## Introduction

In [46]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd
import scipy.stats
from sklearn.cluster import MiniBatchKMeans

In [5]:
from geo3dfeatures import features, extract, postprocess
from geo3dfeatures.tools import kmean

In [35]:
datapath = Path("..", "data")
experiment = "b9"
xyz = True
n_neighbors = 50
radius = None
n_clusters = 3
binsize = 1.0
feature_set = "full"
config_name = "base"
config_path = Path("..", "config", config_name + ".ini")
kd_tree_leaf_size = 100

In [31]:
BATCH_SIZE = 10_000
SEED = 1337

## Calcul des clusters via le k-mean

In [21]:
data = kmean.load_features(datapath, experiment, n_neighbors, radius, feature_set, binsize)

[17:10:50] kmean.load_features (INFO) - Recover features stored in ../data/output/b9/features/features-n50-full-binsize-1.0.csv


In [22]:
data.shape

(22300, 25)

In [23]:
feature_config = kmean.read_config(config_path)

In [24]:
points = data[["x", "y", "z"]].copy()
data.drop(columns=["x", "y"], inplace=True)

In [26]:
for c in data.columns:
    data[c] = features.max_normalize(data[c])

In [27]:
if "bin_z_range" in data.columns:
    data["bin_z_range"].fillna(0, inplace=True)

In [28]:
kmean.update_features(data, feature_config)

In [127]:
model = MiniBatchKMeans(n_clusters=n_clusters, batch_size=BATCH_SIZE, random_state=SEED)
model.fit(data)
labels = model.labels_

## Chargement du kd-tree

In [38]:
tree_file = Path(datapath, "output", experiment, "kd-tree-leaf-" + str(kd_tree_leaf_size) + ".pkl")
with open(tree_file, "rb") as fobj:
    tree = pickle.load(fobj)

## Définition des méthodes de calcul du mode statistique

On cherche ici à trouver le triplet `(r, g, b)` le plus fréquent dans le voisinage de chacun des 22300 points du nuage.

In [128]:
colored_results = kmean.colorize_clusters(points, labels)
colored_values = colored_results.values

### Méthode 1: `scipy.stats.mode()`

Dans cette méthode, on itère sur toutes les lignes du dataframe (avec `df.values`, plus efficace que `df.iterrows()`), et pour chaque point examiné, on fait une requête au kd-tree pour avoir l'ensemble de ses voisins, puis on calcule le mode statistique avec `scipy.stats.mode`, qui renvoie la valeur la plus fréquente dans un tableau à une dimension.

In [99]:
def mode1(df, tree, n_neighbors):
    new_clusters = []
    for point in df.values[:int(df.shape[0] * 0.1)]:  # On ne considère que 10% du nuage de point
        _, neighbors = extract.request_tree(point[:3], tree, n_neighbors, None)
        new_cluster = scipy.stats.mode(df.values[neighbors, 3:]).mode
        new_clusters.append(new_cluster)
    return new_clusters

In [100]:
%timeit mode1(colored_results, tree, n_neighbors)

1.68 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Méthode 2: `groupby().count()` de pandas

Ici, même structure que précédemment, sauf qu'on utilise une construction purement issue de `pandas`, avec un `groupby`, un décompte sur chacun des groupes, et la sélection du groupe majoritaire.

In [102]:
def mode2(df, tree, n_neighbors):
    new_clusters = []
    for point in df.values[:int(df.shape[0] * 0.1)]:  # On ne considère que 10% du nuage de point
        _, neighbors = extract.request_tree(point[:3], tree, n_neighbors, None)
        new_cluster = df.iloc[neighbors].groupby(["r", "g", "b"])["x"].count().idxmax()
        new_clusters.append(new_cluster)
    return new_clusters

In [103]:
%timeit mode2(colored_results, tree, n_neighbors)

5.32 s ± 376 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Méthode 3: `mode()` de pandas

Dans cette troisième méthode, il s'agit toujours d'itérer sur les lignes du dataframe, en appliquant cette fois-ci la méthode `mode()` propre à pandas.

In [104]:
def mode3(df, tree, n_neighbors):
    new_clusters = []
    for point in df.values[:int(df.shape[0] * 0.1)]:  # On ne considère que 10% du nuage de point
        _, neighbors = extract.request_tree(point[:3], tree, n_neighbors, None)
        new_cluster = df.iloc[neighbors, 3:].mode()
        new_clusters.append(new_cluster)
    return new_clusters

In [105]:
%timeit mode3(colored_results, tree, n_neighbors)

7.33 s ± 577 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Méthode 4: adaptation de `scipy.stats.mode()` pour un `np.array` multidimensionnel

Dans cette quatrième méthode, on change de paradigme, et on s'intéresse de plus près à la façon de produire un mode statistique "à la main", sur l'intégralité du jeu de données, en exploitant `numpy`. Cela nous conduit à calculer les voisins en une fois, en amont du process (on profite là de la souplesse de `scipy` pour la manipulation du kd-tree).

In [136]:
def mode4(data, tree, n_neighbors):
    """See https://github.com/scipy/scipy/blob/master/scipy/stats/stats.py#L458
    """
    _, neighbors = extract.request_tree(data[:,:3], tree, n_neighbors, radius)
    arr = data[neighbors, 3:]
    scores = np.unique(np.reshape(arr, (-1, arr.shape[-1])), axis=0)
    mostfrequent = np.squeeze(np.zeros((arr.shape[0], arr.shape[2])))
    oldcounts = np.zeros(arr.shape[0])
    for score in scores:
        template = np.all(arr == score, axis=2)
        counts = np.sum(template, axis=1)
        mostfrequent[counts > oldcounts] = score
        oldcounts = np.maximum(counts, oldcounts)
    return mostfrequent

In [137]:
%timeit mode4(colored_values, tree, n_neighbors)

2.61 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Méthode 5: adaptation de `np.argmax(np.bincount())` pour un `np.array` multidimensionnel

Idem que précédemment, sauf qu'on généralise tout ça en appliquant la même logique que dans la méthode 2, à savoir `np.bincount` associé à `np.argmax`.

In [138]:
def mode5(data, tree, n_neighbors):
    """See https://stackoverflow.com/questions/12297016/how-to-find-most-frequent-values-in-numpy-ndarray
    """
    _, neighbors = extract.request_tree(data[:,:3], tree, n_neighbors, radius)
    arr = data[neighbors, 3:]
    u, indices = np.unique(arr, return_inverse=True)
    return u[np.argmax(np.apply_along_axis(np.bincount, 1, indices.reshape(arr.shape), None, np.max(indices)+1), axis=1)]

In [139]:
%timeit mode5(colored_values, tree, n_neighbors)

1.05 s ± 37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Bilan

En résumé, on note des performances bien supérieures si on traite tout le dataset d'un seul coup, en employant les capacités de `numpy` !

| méthode |       temps       |        commentaire            |
|---------|-------------------|-------------------------------|
| 1       | 1.68s+/-0.105s    | 10% des 22300 points étudiés  |
| 2       | 5.32s+/-0.376s    | 10% des 22300 points étudiés  |
| 3       | 7.33s+/-0.577s    | 10% des 22300 points étudiés  |
| 4       | 2.61s+/-0.169s    |                               |
| 5       | 1.05s+/-0.037s    |                               |


## Etre plus malin, et calculer le mode sur les labels

Jusqu'à présent, les modes statistiques ont été calculés sur trois colonnes, à savoir les 3 canaux RGB. Avant de faire cette transformation, les étiquettes associées au clustering représentent une seule colonne. Calculer des modes sous cette forme doit pouvoir être plus rapide, et moins consommateur en RAM.

In [143]:
labelled_point_cloud = np.append(point_cloud, np.expand_dims(labels, axis=1), axis=1)

In [144]:
%timeit mode5(labelled_point_cloud, tree, n_neighbors)

438 ms ± 8.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


On voit qu'on réduit ainsi encore le temps de calcul de près de deux tiers.